In [1]:
import numpy as np
import tensorflow as tf

2024-08-11 16:15:22.288980: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 16:15:22.298745: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-11 16:15:22.327338: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-11 16:15:22.328023: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 16:15:22.957641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

## Import the Data

In [2]:
train_data = np.load('Audiobooks_train_data.npz')
train_input = train_data['input'].astype(float)
train_target = train_data['target'].astype(int)

validation_data = np.load('Audiobooks_validation_data.npz')
validation_input = validation_data['input'].astype(float)
validation_target = validation_data['target'].astype(int)

test_data = np.load('Audiobooks_test_data.npz')
test_input = test_data['input'].astype(float)
test_target = test_data['target'].astype(int)

train_input.shape

(3579, 10)

## Model

In [3]:
output_layer = 2
hidden_layer = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer,activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer,activation = 'relu'),
                          
                            tf.keras.layers.Dense(output_layer,activation = 'softmax')
    
])

model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

## Train the Model

In [4]:
num_epochs = 100
batchSize = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5)
model.fit(train_input, train_target,
          epochs = num_epochs,
          batch_size=batchSize,
          callbacks = early_stopping,
          validation_data = (validation_input,validation_target),
          verbose = 2)

Epoch 1/100
36/36 - 0s - loss: 0.5295 - accuracy: 0.7144 - val_loss: 0.4043 - val_accuracy: 0.8166 - 464ms/epoch - 13ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4174 - accuracy: 0.7868 - val_loss: 0.3515 - val_accuracy: 0.8345 - 58ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3821 - accuracy: 0.7988 - val_loss: 0.3491 - val_accuracy: 0.8009 - 66ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3670 - accuracy: 0.7994 - val_loss: 0.3249 - val_accuracy: 0.8367 - 65ms/epoch - 2ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3562 - accuracy: 0.8156 - val_loss: 0.3277 - val_accuracy: 0.8277 - 60ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3547 - accuracy: 0.8061 - val_loss: 0.3198 - val_accuracy: 0.8345 - 52ms/epoch - 1ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3457 - accuracy: 0.8209 - val_loss: 0.3191 - val_accuracy: 0.8367 - 53ms/epoch - 1ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3384 - accuracy: 0.8203 - val_loss: 0.3328 - val_accuracy: 0.8233 - 68ms/epoch - 2ms/step
Epoch 9/100
36

## Test Accuracy

In [5]:
test_loss,test_accuracy, = model.evaluate(test_input,test_target)

14/14 [==============================] - 0s 870us/step - loss: 0.3480 - accuracy: 0.8103


In [6]:
print('\nTest loss: {0:0.2f}. Test accuracy: {1:0.2f}%'.format(test_loss,test_accuracy*100.))


Test loss: 0.35. Test accuracy: 81.03%


## Obtain the probability for a customer to convert

In [7]:
model.predict(test_input).round(2)[:,1]

14/14 [==============================] - 0s 832us/step


array([0.  , 1.  , 0.72, 1.  , 0.  , 0.81, 0.34, 0.34, 1.  , 0.  , 0.91,
       0.76, 1.  , 0.78, 1.  , 0.  , 0.  , 0.43, 1.  , 0.69, 0.  , 1.  ,
       0.  , 0.85, 0.  , 0.  , 0.23, 0.41, 1.  , 0.91, 0.  , 0.11, 0.  ,
       1.  , 0.49, 0.46, 0.94, 0.32, 0.  , 0.  , 0.42, 0.  , 0.43, 0.  ,
       0.85, 0.52, 1.  , 0.51, 1.  , 0.63, 0.  , 0.99, 0.34, 0.51, 1.  ,
       0.  , 0.51, 1.  , 0.49, 1.  , 0.84, 0.  , 0.92, 0.82, 0.58, 0.76,
       0.77, 0.56, 0.72, 0.92, 0.  , 0.  , 0.39, 0.97, 0.  , 0.58, 0.37,
       0.  , 1.  , 1.  , 0.  , 0.32, 0.55, 0.17, 0.  , 0.41, 0.47, 0.34,
       1.  , 1.  , 0.34, 0.72, 0.34, 0.46, 0.71, 0.81, 1.  , 0.42, 0.51,
       0.01, 0.  , 0.  , 0.4 , 0.34, 0.95, 0.  , 1.  , 0.85, 0.75, 0.58,
       0.51, 0.56, 0.58, 0.52, 1.  , 1.  , 0.51, 0.99, 1.  , 0.  , 0.58,
       1.  , 0.51, 0.  , 1.  , 0.76, 0.63, 0.46, 0.  , 0.  , 0.87, 1.  ,
       0.  , 0.61, 0.63, 0.  , 0.86, 0.74, 0.64, 0.86, 0.  , 0.  , 0.34,
       0.5 , 0.  , 0.65, 0.  , 0.34, 0.42, 0.98, 0.

In [8]:
np.argmax(model.predict(test_input),axis=1)

14/14 [==============================] - 0s 681us/step


array([0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,

## Save the Model

In [9]:
model.save('audioBooks_model.h5')